In [1]:
import numpy as np
import json
from pymongo import ASCENDING

In [2]:
# %load /home/yqwang/Codes/python/alignment/portrait/DB/MongodbClient.py

import sys,os

from pymongo import MongoClient

class MongodbClient(object):

	def get_db(self):
		return self.db[self.tab]

	def __init__(self, db_host='localhost', db_port=27017, db_name='null', tab_name='null'):
		self.setDatabase(db_host, db_port, db_name, tab_name)

	def setDatabase(self, db_host, db_port, db_name, tab_name):
		self.client = MongoClient(db_host, db_port)
		self.db_host = db_host
		self.db_port = db_port
		self.db_name = db_name
		self.db = self.client[db_name]
		self.tab = tab_name

	def changeTable(self, name):
		self.tab = name

	def put_many(self, records):
		if not isinstance(records, list):
			return
		self.db[self.tab].insert_many(records)

	def put(self, key, dbComm):
		if not isinstance(dbComm, dict):
			return None
		if self.db[self.tab].find_one(key):
			return None
		else:
			self.db[self.tab].insert(dbComm)

	def update(self, key, val, multi=False):
		if not isinstance(val, dict):
			return
		if not self.db[self.tab].find_one(key):
			self.put(key, val)
		else:
			self.db[self.tab].update(key,{"$set":val}, multi=multi)

	def delete(self, key):
		self.db[self.tab].remove(key)

	def search(self, prop_name, prop_vals, search_opera='$in'):
		"""
		For example: search('uid', weibo_uids, '$in') # get user infos from weibodb
		"""
		for p in self.db[self.tab].find({prop_name:{search_opera:prop_vals}}):
			yield p

	def get(self, key):
		if not self.db[self.tab].find_one(key):
			return None
		else:
			return self.db[self.tab].find_one(key)

	def getAll(self, limit=None, skip=0, filter=None):
		if not limit:
			for p in self.db[self.tab].find(filter).skip(skip):
				yield p
		else:
			for p in self.db[self.tab].find(filter).limit(limit).skip(skip):
				yield p

	def clean(self):
		self.client.drop_database(self.db_name)

	def deleteAll(self):
		self.db[self.tab].remove()

In [3]:
!ls

cand.flickr-lastfm.half.feat.p10
cand.flickr-myspace.half.feat.p10
cand.online-offline.half.feat.p10
data
evals_tune_params.sh
eval_tune_params.sh
filename_mv.sh
log
model_res.flickr-myspace.half.feat.p10
model_res.flickr-myspace.half.feat.p10.m0.end
model_res.flickr-myspace.half.feat.p10.m0.src
model_res.flickr-myspace.half.feat.p10.m1.end
model_res.flickr-myspace.half.feat.p10.m1.src
model_res.flickr-myspace.half.feat.p10.m2.end
model_res.flickr-myspace.half.feat.p10.m2.src
model_res.flickr-myspace.half.feat.p10.m3.end
model_res.flickr-myspace.half.feat.p10.m3.src
model_res.flickr-myspace.half.feat.p10.m4.end
model_res.flickr-myspace.half.feat.p10.m4.src
model_res.flickr-myspace.half.feat.p10.m5.end
model_res.flickr-myspace.half.feat.p10.m5.src
model_res.flickr-myspace.half.feat.p10.m6.end
model_res.flickr-myspace.half.feat.p10.m6.src
model_res.flickr-myspace.half.feat.p10.m7.end
model_res.flickr-myspace.half.feat.p10.m7.src
model_res.flickr-myspace.half.feat.p10.m8.end
model_res.fli

In [4]:
def load_modelres(filepath):
    model_res = dict()
    with open(filepath, 'r') as fin:
        for ln in fin:
            model_res = json.loads(ln.strip())
    return model_res

In [5]:
db = MongodbClient(db_name='flickr-myspace')
base_path = 'model_res.flickr-myspace.half.feat.p10'
for i in range(10):
    for tag in ['src', 'end']:
        model_res = load_modelres('model_res.flickr-myspace.half.feat.p10.m%d.%s'%(i, tag))
        db.changeTable('model_res_m%d_%s'%(i, tag))
        code_len = 0
        res_list = list()
        for k,code in model_res.items():
            res = {"nd_id":k}
            for ci in range(len(code)):
                res[str(ci)] = code[ci]
            res_list.append(res)
            code_len = len(code)
        db.put_many(res_list)
        db.db['model_res_m%d_%s'%(i, tag)].create_index("nd_id")
        for k in range(code_len):
            db.db['model_res_m%d_%s'%(i, tag)].create_index(str(k))

KeyboardInterrupt: 

### Anchors在节点间的距离度量上更为相近么？

In [6]:
with open('data/flickr-myspace/flickr-myspace.anchors') as fin:
    anchors = list()
    for ln in fin:
        anchors.append(ln.strip().split(','))

In [7]:
anchors[:10]

[['5611', '8247'],
 ['4730', '10125'],
 ['6371', '4405'],
 ['2555', '9583'],
 ['4362', '1157'],
 ['5625', '6851'],
 ['3332', '7397'],
 ['5253', '1508'],
 ['6568', '3117'],
 ['6359', '1885']]

In [8]:
def get_code(res):
    return  ','.join([str(res[str(i)]) for i in range(14)])

In [9]:
db = MongodbClient(db_name='flickr-myspace')
db.changeTable('model_res_m%d_%s'%(0, 'src'))
print('src')
print('5611',get_code(db.get({'nd_id':'5611'})))
print('4730', get_code(db.get({'nd_id':'4730'})))
print('6371', get_code(db.get({'nd_id':'6371'})))
print('5', get_code(db.get({'nd_id':'5'})))

print('end')
db.changeTable('model_res_m%d_%s'%(0, 'end'))
print('8247', get_code(db.get({'nd_id':'8247'})))
print('10125', get_code(db.get({'nd_id':'10125'})))
print('10125', get_code(db.get({'nd_id':'5611'})))


src
5611 1,-1,1,-1,-1,1,1,-1,-1,1,-1,1,-1,1
4730 1,-1,1,-1,-1,1,1,-1,-1,1,-1,1,-1,1
6371 1,-1,1,-1,-1,1,1,-1,-1,1,-1,-1,-1,1
5 1,-1,1,-1,-1,1,1,-1,-1,1,-1,1,-1,1
end
8247 1,-1,1,-1,-1,1,1,-1,-1,1,-1,-1,-1,1
10125 1,-1,1,-1,-1,1,1,-1,-1,1,-1,1,-1,1
10125 1,1,-1,-1,1,-1,-1,1,-1,1,1,1,-1,1


* anchors内部的距离和外部的距离之间的比较

In [10]:
def code_trans(res, n_dim):
    code = list()
    for i in range(n_dim):
        code.append(res[str(i)])
    return code

In [15]:
def code_sim(from_nd, to_nd):
    hm_dist = 0
    for k in range(len(from_nd)):
#         print(from_nd, to_nd)
        hm_dist += abs(from_nd[k]+to_nd[k])
    return len(from_nd)-hm_dist/2.

In [40]:
lookups = {
    'src': json.load(open('data/flickr-myspace/flickr.node.lookup.json')), 
    'end': json.load(open('data/flickr-myspace/myspace.node.lookup.json'))
}

In [53]:
lookbacks = {
    'src': list(),
    'end': list()
}
tags = ['src', 'end']
idx = 0
for t in ['flickr', 'myspace']:
    lookbacks[tags[idx]] = np.load('data/flickr-myspace/{}.node.lookback.npy'.format(t))
    idx+=1

In [26]:
src_nds_ac = set()
end_nds_ac = set()
for [ac_src, ac_end] in anchors:
    src_nds_ac.add(ac_src)
    end_nds_ac.add(ac_end)

In [71]:
model_res = [{
    'src': list(),
    'end': list()
} for i in range(10)]

for k in range(10):
    for t in tags:
        cnt = 0
        for nd in lookbacks[t]:        
            db.changeTable('model_res_m%d_%s'%(k, t))
            model_res[k][t].append(code_trans(db.get({'nd_id':nd}),14))
            cnt += 1
            if not cnt%1000:
                print('Having processed %d records @%s, model=%d'%(cnt, t, k))

Having processed 1000 records @src, model=0
Having processed 2000 records @src, model=0
Having processed 3000 records @src, model=0
Having processed 4000 records @src, model=0
Having processed 5000 records @src, model=0
Having processed 6000 records @src, model=0
Having processed 1000 records @end, model=0
Having processed 2000 records @end, model=0
Having processed 3000 records @end, model=0
Having processed 4000 records @end, model=0
Having processed 5000 records @end, model=0
Having processed 6000 records @end, model=0
Having processed 7000 records @end, model=0
Having processed 8000 records @end, model=0
Having processed 9000 records @end, model=0
Having processed 10000 records @end, model=0
Having processed 1000 records @src, model=1
Having processed 2000 records @src, model=1
Having processed 3000 records @src, model=1
Having processed 4000 records @src, model=1
Having processed 5000 records @src, model=1
Having processed 6000 records @src, model=1
Having processed 1000 records @

In [73]:
print(len(model_res[0]['end']), len(lookbacks['end']))

10733 10733


In [80]:
hm_dist = [list() for k in range(10)]
hm_dist_ac = [list() for k in range(10)]

for k in range(10):
    for ac_src in src_nds_ac:
        tmp_dist = list()
        tmp_dist_ac = list()
        for j in range(len(lookbacks['end'])):
            if lookbacks['end'][j] not in end_nds_ac:
                tmp_dist.append(code_sim(model_res[k]['src'][lookups['src'][ac_src]], model_res[k]['end'][j]))
            else:
                tmp_dist_ac.append(code_sim(model_res[k]['src'][lookups['src'][ac_src]], model_res[k]['end'][j]))
        hm_dist[k].append(tmp_dist)
        hm_dist_ac[k].append(tmp_dist_ac)

mean_dist = list()
std_dist = list()
mean_dist_ac = list()
std_dist_ac = list()

for k in range(10):
    mean_dist.append(np.mean(hm_dist[k],axis=1))
    std_dist.append(np.std(hm_dist[k],axis=1))
    mean_dist_ac.append(np.mean(hm_dist_ac[k],axis=1))
    std_dist_ac.append(np.std(hm_dist_ac[k],axis=1))

In [81]:
print(mean_dist[0][0],std_dist[0][0])

12.727594114274794 1.4561023898314922


In [82]:
print(mean_dist_ac[0][0],std_dist_ac[0][0])

2.606741573033708 3.525035057157486


In [89]:
dist_gap = tuple()
for i in range(len(mean_dist)):
    dist_gap += mean_dist[i]-mean_dist_ac[i],

In [93]:
print(dist_gap[0])

[10.12085254 10.12085254 10.12085254 10.12085254 11.15616503 10.12085254
 11.15616503 11.15616503 11.15616503 11.15616503 11.15616503 10.12085254
 10.12085254 11.15616503 11.15616503 10.12085254 10.12085254 10.12085254
 10.12085254 11.15616503 11.15616503 10.12085254 10.12085254 10.12085254
 11.15616503 11.15616503 10.12085254 10.12085254 11.15616503 10.12085254
 10.12085254 11.15616503 10.12085254 10.12085254 10.12085254 10.12085254
 11.15616503 10.12085254 10.12085254 11.15616503 10.12085254 11.15616503
 10.12085254 10.12085254 10.12085254 10.12085254 10.12085254 10.12085254
 10.12085254 11.15616503 11.15616503 10.12085254 11.15616503 11.15616503
 10.12085254 10.12085254 10.12085254 11.15616503 10.12085254 11.15616503
 11.15616503 11.15616503 10.12085254 11.15616503 10.12085254 10.12085254
 11.15616503 11.15616503 10.12085254 11.15616503 10.12085254 11.15616503
 10.12085254 10.12085254 10.12085254 10.12085254 11.15616503 10.12085254
 11.15616503 10.12085254 10.12085254 10.12085254 10

* anchors在训练集内部的和在anchors内部的距离比较

In [83]:
with open('data/flickr-myspace/train-test/flickr-myspace.anchors.ptrain10.train') as fin:
    anchors_train = list()
    for ln in fin:
        anchors_train.append(ln.strip().split(','))
        
src_nds_ac_train = set()
end_nds_ac_train = set()
for [ac_src, ac_end] in anchors:
    src_nds_ac_train.add(ac_src)
    end_nds_ac_train.add(ac_end)

In [84]:
hm_dist_ac_train = [list() for k in range(10)]

for k in range(10):
    for ac_src in src_nds_ac_train:
        tmp_dist_ac = list()
        for j in range(len(lookbacks['end'])):
            if lookbacks['end'][j] in end_nds_ac_train:
                tmp_dist_ac.append(code_sim(model_res[k]['src'][lookups['src'][ac_src]], model_res[k]['end'][j]))
        hm_dist_ac_train[k].append(tmp_dist_ac)

In [86]:
mean_dist_ac_train = list()
std_dist_ac_train = list()

for k in range(10):
    mean_dist_ac_train.append(np.mean(hm_dist_ac_train[k],axis=1))
    std_dist_ac_train.append(np.std(hm_dist_ac_train[k],axis=1))

In [87]:
dist_gap_ac = tuple()
for i in range(len(mean_dist_ac)):
    dist_gap_ac += mean_dist_ac[i]-mean_dist_ac_train[i],

In [92]:
print(dist_gap_ac[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


### 哪些节点之间的距离度量比较相近？